In [1]:
import ee
ee.Initialize()

In [2]:
import geepy
import math
import sys

In [3]:
# Teste 1:

# Todas as classes todo o periodo
# Floresta Período chuvoso

config = geepy.params.configParams('input_classification_v30.json')

ls = ee.FeatureCollection(config.params['lsTiles'])
tiles = [geepy.feature.getTiles(ls, tile, 'TILE_T') for tile in config.params['grid']['tiles2process']]

# amostras = ee.FeatureCollection(config.params['samples']['2016v8'])

watersheds = ee.FeatureCollection(config.params['studyArea']) 

# pivots = {i: ee.FeatureCollection(config.params['pivots'] + i) for (i) in config.params['years2process']}

# bands = config.params['bandParams']

l5 = ee.ImageCollection(config.params['imgCollection']['lc5']['id']).select(
    config.params["imgCollection"]["lc5"]["bands"],
    config.params["imgCollection"]["lc5"]["bandNames"])

l7 = ee.ImageCollection(config.params['imgCollection']['lc7']['id']).select(
    config.params["imgCollection"]["lc7"]["bands"],
    config.params["imgCollection"]["lc7"]["bandNames"])

l8 = ee.ImageCollection(config.params['imgCollection']['lc8']['id']).select(
    config.params["imgCollection"]["lc8"]["bands"],
    config.params["imgCollection"]["lc8"]["bandNames"])

In [4]:
tileNames = [tiles[i][0] for i in range(len(tiles))]
tileCoords = [tiles[i][1].geometry().getInfo()['coordinates'] for i in range(len(tiles))]

In [5]:
def clipImage(img, tile):
    """
    clipImageCollection docstring
    
    Args:
        imgCollection:
        tile:
    """
    return img.clip(tile)

In [6]:
centroids = [tiles[i][1].map(ee.Feature.centroid) for i in range(len(tiles))]

In [7]:
def sumGoodPixels(img, centroid):
    img = img.filterBounds(centroid).sum()
    return img

In [8]:
def remapCloudMask(img):
    #return img.select('cloudmask').remap([0,1],[1,0])
    return img.addBands(img.select('cloudmask').remap([0,1],[1,0]))

In [9]:
landsat = {}
goodPixels = {}
mosaic = {}

goodPixelsMosaic = {}

for year in config.params['years2process']:  
    start_d = year + config.params['period']['dry']['start']
    end_d = year + config.params['period']['dry']['end']


    #print("Start-End dry season %s %s" %(start_d, end_d))

    if(int(year) < 2002):
        filtered = l5.filterMetadata('CLOUD_COVER', 'less_than', config.params['cloudCoverThreshold']).filterDate(start_d, end_d).map(geepy.image.maskLandsatSR)
        satellite = 'l5'
    elif(int(year) in (2002, 2011, 2012)):
        filtered = l7.filterMetadata('CLOUD_COVER', 'less_than', config.params['cloudCoverThreshold']).filterDate(start_d, end_d).map(geepy.image.maskLandsatSR)
        satellite = 'l7'
    elif(int(year) > 2002 and int(year) < 2011):
        filtered = l5.filterMetadata('CLOUD_COVER', 'less_than', config.params['cloudCoverThreshold']).filterDate(start_d, end_d).map(geepy.image.maskLandsatSR)
        satellite = 'l5'
    else:
        filtered = l8.filterMetadata('CLOUD_COVER', 'less_than', config.params['cloudCoverThreshold']).filterDate(start_d, end_d).map(geepy.image.maskLandsatSR)
        satellite = 'l8'
    
    filtered = filtered.map(remapCloudMask)
    
    landsat[year] = [sumGoodPixels(filtered, centroids[i]) for i in range(len(centroids))]
    
    #goodPixels[year] = [clipImage(landsat[year][i], tiles[i][1]) for i in range(len(tiles))]
    
    goodPixels[year] = [landsat[year][i] for i in range(len(tiles))]
    
    mosaic[year] = ee.ImageCollection(goodPixels[year]).mosaic().clip(watersheds).multiply(10000).toByte()
    
    goodPixelsMosaic[year] = mosaic[year].select('cloudmask')
    
    sys.stdout.write("\rProcessing Landsat data: %s" % year)
    sys.stdout.flush()

Processing Landsat data: 2018

In [10]:
mapa = geepy.maps.geeMap(watersheds, zoom=10)

# Training

# 1 Flormações florestais -------------------------------- green
# 2 Formações savânicas ---------------------------------- lightgreen
# 3 Agricultura de Sequeiro ------------------------------ pink
# 4 Agricultura Irrigada --------------------------------- magenta
# 5 Pastagem --------------------------------------------- yellow
# 6 Corpos d'água ---------------------------------------- blue
# 7 Área urbana/Construções rurais ----------------------- red

viz_params = {'min': 0, 'max': 24, 'palette':'#004000, #34f200, #ffcaff, #ff42f9, #f4f383, #0000ff, #ff0000','format':'png'}


mapa.addLayer(goodPixelsMosaic['1990'], viz_params=viz_params, name='cm')
mapa.addLayer(tiles[0][1], name='ts')
mapa.addControls()

mapa.show()

In [11]:
coords = [[[-46.632916356914535, -15.447083782725066],\
   [-43.13041651144095, -15.447083782725066],\
   [-43.13041651144095, -10.181249376641395],\
   [-46.632916356914535, -10.181249376641395],\
   [-46.632916356914535, -15.447083782725066]]]

In [13]:
# To drive
tasks = {year: geepy.image.send2drive(goodPixelsMosaic[year], coords, 'goodPixels'+year+'_v30', 'goodPixels_v30', 30) for year in config.params['years2process']}

In [14]:
for i in tasks.keys():
   [tasks[i].start()]